In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from datetime import datetime, timedelta
import pandas as pd

# current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
current_date = pd.to_datetime(datetime(2022, 11, 30, 0, 0, 0))
print(f'{current_date=}')
# current_date = pd.Timestamp('2023-02-28 09:00:00')

current_date=Timestamp('2022-11-30 00:00:00')


In [2]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()

Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.4) by running 'pip install hopsworks==3.4.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/413849
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.4) by running 'pip install hopsworks==3.4.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/413849
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)
features

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.4) by running 'pip install hopsworks==3.4.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/413849
Connected. Call `.close()` to terminate connection gracefully.
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/413849
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using ArrowFlight (11.40s) 


,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id
0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,...,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,2022-11-30,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-11-30,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-11-30,3
3,7.0,5.0,0.0,0.0,1.0,0.0,4.0,2.0,11.0,7.0,...,3.0,3.0,1.0,4.0,3.0,1.0,6.0,6.0,2022-11-30,4
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-11-30,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,4.0,4.0,2.0,2.0,0.0,1.0,7.0,8.0,22.0,15.0,...,41.0,31.0,34.0,29.0,22.0,19.0,29.0,3.0,2022-11-30,261
261,7.0,4.0,1.0,3.0,7.0,19.0,83.0,129.0,117.0,128.0,...,123.0,138.0,113.0,77.0,50.0,39.0,17.0,13.0,2022-11-30,262
262,35.0,18.0,8.0,5.0,6.0,22.0,71.0,100.0,123.0,127.0,...,118.0,136.0,133.0,138.0,117.0,109.0,88.0,61.0,2022-11-30,263
263,17.0,8.0,8.0,1.0,7.0,5.0,24.0,38.0,66.0,60.0,...,111.0,165.0,111.0,112.0,109.0,78.0,126.0,93.0,2022-11-30,264


In [5]:
predictions = get_model_predictions(model, features)
predictions['pickup_hour'] = current_date
predictions

[LightGBM] [Warning] feature_fraction is set=0.7349238836920697, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7349238836920697
[LightGBM] [Warning] bagging_fraction is set=0.9941706329137243, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9941706329137243


,pickup_location_id,predicted_demand,pickup_hour
0,1,0.0,2022-11-30
1,2,0.0,2022-11-30
2,3,0.0,2022-11-30
3,4,4.0,2022-11-30
4,5,0.0,2022-11-30
...,...,...,...
260,261,3.0,2022-11-30
261,262,6.0,2022-11-30
262,263,34.0,2022-11-30
263,264,56.0,2022-11-30


In [6]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/413849/fs/409672/fg/537930


Uploading Dataframe: 0.00% |          | Rows 0/265 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/413849/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x264e91dd950>, None)